In [ ]:
# Задание 1
# Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, 
# которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением 
# столбца timestamp для данного значения userId.

In [ ]:
import pandas as pd
ratings = pd.read_csv('ml-latest-small/ratings.csv', sep = ',')
ratings.head()

In [ ]:
# отфильтруем данные и выведем их в отдельный Df
# фильтруем данные, группируем, считаем по каджому его количество и сбрасываем индекс в новом df

user_ratings = ratings[['userId', 'movieId']]
user_count_ratings = user_ratings.groupby('userId').count().reset_index()

# переименуем столбец 'movieId' в 'count_ratings' и отфильтруем его с количеством более 100

user_count_ratings.rename(columns={'movieId': 'count_ratings'}, inplace=True)
user_count_ratings = user_count_ratings.query('count_ratings > 100')
user_count_ratings

In [ ]:
# производим группировку, посчитаем  min|max timestamp-а, сбросим индексы для 
# корректного отображения таблицы и создадим новый столбец с разницой tampstamp-ов и назовем его 'timelife'

timestamp = ratings.groupby('userId').agg([min, max]).reset_index()
timestamp['timelife'] = timestamp['timestamp']['max'] - timestamp['timestamp']['min']
timestamp_final = timestamp[['userId', 'timelife']]
timestamp_final.head()

In [ ]:
# соединяем две таблицы

user_count_ratings.merge(timestamp_final, how = 'left', on = 'userId')

In [ ]:
# расчет средней и визуал

round_ = round(timestamp['timelife'].mean()/(60*60*24))
print(f'Cреднее время жизни пользователей, которые выставили более 100 оценок: {round_}')
timestamp.agg({'timelife': ['min', 'max', 'mean', 'median']}).round()

In [ ]:
# Задание 2
# Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
# таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
# аналогичную таблицу по типам выручки с указанием адреса клиента
# Обратите внимание, что в процессе объединения таблиц данные не должны теряться.
# Дана статистика услуг перевозок клиентов компании по типам:
# rzd - железнодорожные перевозки
# auto - автомобильные перевозки
# air - воздушные перевозки
# client_base - адреса клиентов

In [ ]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

In [ ]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

In [ ]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

In [ ]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base_id = client_base[['client_id']]
client_base_full = client_base

In [ ]:
# без адреса

(
client_base_id.merge(rzd, how = 'outer', on = 'client_id')
    .merge(auto, how = 'outer', on = 'client_id')
    .merge(air, how = 'outer', on = 'client_id')
)

In [ ]:
# с адресом

(
client_base_full.merge(rzd, how = 'outer', on = 'client_id')
    .merge(auto, how = 'outer', on = 'client_id')
    .merge(air, how = 'outer', on = 'client_id')
)

In [ ]:
# Задание 3

# В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна 
# последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили 
# эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:

# 1. У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется 
# одно-два числа. Как их получить?
# 2. Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе 
# расчетах пункта 1?
# 3. Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

# Решение
# 1. merge по ID пользователя - соединение координат и общих данных. Данные timestamp - разделить покупки и визиты с координатами.
# timestamp (timestamp визита|timestamp покупки) и ID - связываем все данные.
# 2. Не поняла вопроса :(
# 3. По координатам пользователей:
# - в каких регионах имеется повышенный спрос (или наоборот - низкий спрос) на продукцию компании
# - как можно оптимизировать логистику и направлении организации 
# - ответить на вопросы: где-то необходимо открыть|закрыть офисы продаж, перенаправить тот или иной товар в другой регион